In [4]:
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from datetime import datetime
import pandas as pd

def make_scatter(x, y, cats, n_points=1000):
    cat_lut = {
        1: "Nothing",
        2: "Beer",
        3: "Wine",
        4: "Liquor (Drinks)",
        5: "Liquor (Shots)"
    }
    color_lut = {
        "Nothing": "#0F0E0E",
        "Beer": "#f28e1c",
        "Wine": "#BA091E",
        "Liquor (Drinks)": "#ADD8E6",
        "Liquor (Shots)": "#2364AA"
    }

    xx, yy, cc = [], [], []
    for idx, ((x0, x1), (y0, y1)) in enumerate(zip(zip(x[:-1], x[1:]), zip(y[:-1], y[1:]))):
        points = int(n_points*((x1 - x0)/(max(x) - min(x))))
        xx.extend(pd.to_datetime(np.linspace(pd.Timestamp(x0).value, pd.Timestamp(x1).value, points)))
        yy.extend(np.linspace(y0, y1, points))
        colors = [val for val in list(cats[idx]) for _ in range(int(points/len(cats[idx])))]
        colors = colors + [cats[idx][-1]]*(points - len(colors))
        cc.extend(colors)
        
    cc = [cat_lut[e] if e in cat_lut else e for e in cc]

    fig = px.scatter(x=xx, y=yy, color=cc,
                    color_discrete_map=color_lut)
    return fig

In [32]:
import numpy as np
import pandas as pd
df = pd.read_csv("jkr5famt_04-10-2022,_12-58-14PM.csv").drop(columns="Unnamed: 0")
user_dfs = df.groupby('user')
mdf = pd.DataFrame()
mdf['dt'] = df['dt'].unique()
mdf = mdf.set_index('dt').sort_index()
for username, udf in user_dfs:
    udf = udf.set_index("dt")
    mdf[username+'_bac'] = udf['bac']
mdf = mdf.fillna(method="ffill").fillna(method="bfill")
mdf = mdf.mean(axis=1).reset_index()
mdf.columns = ["Datetime", "Party Average"]
mdf['Threshold'] = 0.08
df = df.rename({'dt': 'Datetime', 'bac': 'BAC', 'user': "User"}, axis=1)

In [55]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.line(df, x='Datetime', y='BAC', color='User')
fig.add_trace(go.Scatter(x=mdf['Datetime'], y=mdf['Party Average'], name='Party Average',
                         line=dict(color='rgba(85, 111, 230, 0.5)', width=4, dash='dot'), mode='lines'))
fig.add_trace(go.Scatter(x=mdf['Datetime'], y=mdf['Threshold'], name='Failure Threshold',
                         line=dict(color='rgba(255, 25, 0, 0.2)', width=4, dash='dash'), mode='lines'))
fig.show()